In [6]:
import psycopg2
from pathlib import Path
import os

os.chdir("..")
from modules.utils.database_utils import get_db_connection

def add_clusters_table():
    """Add clusters and cluster_points tables to existing database."""
    
    # Use your existing connection function
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # SQL commands directly in the function
    sql_commands = """
    CREATE TABLE IF NOT EXISTS clusters (
        cluster_id SERIAL PRIMARY KEY,
        parent_cluster_id INTEGER REFERENCES clusters(cluster_id),
        title VARCHAR(255),
        summary TEXT,
        layer INTEGER,
        created_at TIMESTAMPTZ DEFAULT NOW(),
        filters_used JSONB,
        config JSONB DEFAULT '{}'::jsonb
    );

    CREATE TABLE IF NOT EXISTS cluster_points (
        cluster_id INTEGER REFERENCES clusters(cluster_id) ON DELETE CASCADE,
        point_id BIGINT REFERENCES point(point_id) ON DELETE CASCADE,
        PRIMARY KEY (cluster_id, point_id)
    );

    -- Add indexes for performance
    CREATE INDEX IF NOT EXISTS idx_clusters_parent ON clusters(parent_cluster_id);
    CREATE INDEX IF NOT EXISTS idx_clusters_layer ON clusters(layer);
    CREATE INDEX IF NOT EXISTS idx_cluster_points_cluster ON cluster_points(cluster_id);
    CREATE INDEX IF NOT EXISTS idx_cluster_points_point ON cluster_points(point_id);
    CREATE INDEX IF NOT EXISTS idx_clusters_created_at ON clusters(created_at);
    CREATE INDEX IF NOT EXISTS idx_clusters_filters ON clusters USING GIN (filters_used);
    CREATE INDEX IF NOT EXISTS idx_clusters_config ON clusters USING GIN (config);
    """
    
    try:
        print("Connected to database successfully")
        
        # Execute the SQL commands
        cursor.execute(sql_commands)
        conn.commit()
        
        print("✅ Successfully created clusters tables and indexes!")
        
        # Verify tables were created
        cursor.execute("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public' 
            AND table_name IN ('clusters', 'cluster_points')
            ORDER BY table_name;
        """)
        
        tables = cursor.fetchall()
        print(f"Tables created: {[table[0] for table in tables]}")
        
        # Check indexes
        cursor.execute("""
            SELECT indexname 
            FROM pg_indexes 
            WHERE tablename IN ('clusters', 'cluster_points')
            ORDER BY indexname;
        """)
        
        indexes = cursor.fetchall()
        print(f"Indexes created: {[idx[0] for idx in indexes]}")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

# Run it
add_clusters_table()

Connected to database successfully
✅ Successfully created clusters tables and indexes!
Tables created: ['cluster_points', 'clusters']
Indexes created: ['cluster_points_pkey', 'clusters_pkey', 'idx_cluster_points_cluster', 'idx_cluster_points_point', 'idx_clusters_config', 'idx_clusters_created_at', 'idx_clusters_filters', 'idx_clusters_layer', 'idx_clusters_parent']
